In [1]:
!pip install transformers==3.5.1

Per prima cosa importo da pachetto hugginface Bert il suo tokenizer WordPiece e pytorch

In [2]:
from transformers import BertModel, BertTokenizer



/home/matteo/PycharmProjects/Longformer/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import torch


Successicvamente dobbiamo scaricare il modello Bert, scegliamo il modello base con 12 encoder, 12 teste di attenzione
per layer e 764 unità di attivazione per ogni rete feedward per gli encoder.
chiamiamo il metodo from_pretrained di BertModel e selezionare il modello base passiamoli la stringa "bert-base-uncased"

In [4]:
model = BertModel.from_pretrained("bert-base-uncased")

lo stesso facciamo per il tokenizer


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Ora, vediamo come preelaborare l'input prima di inviarlo a BERT.

In [6]:
sentence = "I love Paris"

In [7]:
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'paris']


Ora aggiungiamo i token speciali, quello  di classificazione [CLS] e quello di fine frase [SEP]



In [8]:
tokens = ['[CLS]']+tokens+['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


La lista dei tokens è 5, ma supponiamo che dobbiamo avere un numero di tokens fisso diciamo che la lista
deve essere lunga 7. Per raggiungere questo obbiettivo possiamo pensare di riempirla con un token speciale
che come unico scopo sia solo quello di riempire la lista dei token, [PAD].

In [9]:
tokens = tokens+ ['[PAD]']+ ['[PAD]']

In [10]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


A questo punto dobbiamo dire al nostro modello quali sono i token reali e quali no ([PAD]),
per fare questo dobbiamo definire un oggetto chiamato attention mask (da non confondere per la matrice di attenzione)
Attention mask assegna 1 ai token diversi da [PAD] e 0 a [PAD]

In [11]:
#attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
attention_mask = [1 if token != '[PAD]' else 0 for token in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


Ora facciamo un'altro passo che nella teoria non viene detto.
I tokens non possono rimanere parole, ma devono essere convertite in numeri perchè gli algoritmi lavorano
su questo.

In [12]:
tokens_id = tokenizer.convert_tokens_to_ids(tokens)
print(tokens_id)

[101, 1045, 2293, 3000, 102, 0, 0]


Dall'output, possiamo osservare che ogni token è mappato a un ID token univoco.
Il prossimo passo è convertite la maschera di attenzione e la lista tokens_id in tensori con pytorch
si consiglia di portarli alla gpu
Si raccomanda di fare unsqueeze(0) per aumentare le dimensioni del tensore, in quanto la semplice lista
non permette di rappresentare un vettore riga che è caratterizzato da 2 dimensioni (riga, colonna),
senza unsqueeze(0) non avremmo la rappresentazione algebrica in quanto sarebbe su un unica dimensione e rappresentebbe
solo il numero di elementi che compongono il tensore che in questo caso sarebbe solo 7

In [13]:
token_ids = torch.tensor(tokens_id).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

Ora dobbiamo ottenere gli embedding.
Il nostro Bert restitusice due oggetti, analizziamoli:
hidden_rep consiste nella rappresentazione finale del bert le famose R_CLS, R_I, R_love, R_Paris, R_SEP, R_PAD.
Infatti la grandezza del tensore è [1,7, 768], 1 perchè abbiamo una frase, 7, perchè abbiamo 7 token e 768
perchè l'ultimo encoder di BERT (ma come tutti gli encoder al suo interno) ha una rete feedforward con 768 unità
che altro non rappresentano le dimensioni di embedding di ogni singolo token.
Infine abbiamo cls_head che altro non è la rappresentazione finale del token speciale di classificazione R_CLS ,
che conterrà l'intera informazione aggregata della frase I love Paris, la sua dimensione è [1,768]


In [14]:
hidden_rep, cls_head = model(token_ids, attention_mask)

In [15]:
print("hidden_rep: \n {}".format(hidden_rep))
print("cls_head: \n {}".format(cls_head))


hidden_rep: 
 tensor([[[-0.0719,  0.2163,  0.0047,  ..., -0.5865,  0.2262,  0.1981],
         [ 0.2236,  0.6536, -0.2294,  ..., -0.3547,  0.5517, -0.2367],
         [ 1.0410,  0.7755,  1.0335,  ..., -0.5621,  0.5218, -0.0852],
         ...,
         [ 0.6156,  0.1036, -0.1875,  ..., -0.3799, -0.7008, -0.3500],
         [ 0.0791,  0.4287,  0.4147,  ..., -0.2417,  0.2403,  0.0378],
         [-0.0165,  0.2459,  0.4566,  ..., -0.2179,  0.1876,  0.0228]]],
       grad_fn=<NativeLayerNormBackward>)
cls_head: 
 tensor([[-9.0660e-01, -3.4189e-01, -3.3729e-01,  7.7140e-01,  6.0975e-02,
         -1.0525e-01,  9.0143e-01,  2.5822e-01, -2.7881e-01, -9.9997e-01,
         -1.0322e-01,  7.4772e-01,  9.8521e-01,  5.9799e-02,  9.4447e-01,
         -5.9859e-01, -2.0539e-01, -5.7386e-01,  3.7684e-01, -7.5183e-01,
          6.6604e-01,  9.9584e-01,  4.2231e-01,  2.2824e-01,  4.9139e-01,
          9.2378e-01, -6.6123e-01,  9.3111e-01,  9.6157e-01,  6.8816e-01,
         -6.7706e-01,  1.2696e-01, -9.8735e-01

In [16]:
print("hidden_rep shape: \n {}".format(hidden_rep.size()))
print("cls_head shape: \n {}".format(cls_head.size()))

hidden_rep shape: 
 torch.Size([1, 7, 768])
cls_head shape: 
 torch.Size([1, 768])


In [17]:
print("hidden_rep num elementi: \n {}".format(torch.numel(hidden_rep)))
print("cls_head num elementi: \n {}".format(torch.numel(cls_head)))

hidden_rep num elementi: 
 5376
cls_head num elementi: 
 768


E' possibile ottenere anche i valori delle singole rappresentazioni da hidden_rep

In [18]:
#hidden_rep[0][0] #R_CLS
#hidden_rep[0][1] #R_I
#hidden_rep[0][2] #R_love

Possiamo anche estrarre gli incorporamenti da tutti gli strati dell'encoder di BERT? Sì!
Vediamo come, dobbiamo in pratica rifare da capo tutto!
Per prima cosa quando scarichiamo il modello bert dobbiamo impostare a true un valore di flag
chiamato output_hidden_states = True

In [19]:
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [21]:
sentence = "I love Paris"

In [22]:
tokens = tokenizer.tokenize(sentence)
print(tokens)

['i', 'love', 'paris']


Ora aggiungiamo i token speciali, quello  di classificazione [CLS] e quello di fine frase [SEP]


In [23]:
tokens = ['[CLS]']+tokens+['[SEP]']
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


La lista dei tokens è 5, ma supponiamo che dobbiamo avere un numero di tokens fisso diciamo che la lista
deve essere lunga 7. Per raggiungere questo obbiettivo possiamo pensare di riempirla con un token speciale
che come unico scopo sia solo quello di riempire la lista dei token, [PAD].

In [24]:
tokens = tokens+ ['[PAD]']+ ['[PAD]']

In [25]:
print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


A questo punto dobbiamo dire al nostro modello quali sono i token reali e quali no ([PAD]),
per fare questo dobbiamo definire un oggetto chiamato attention mask (da non confondere per la matrice di attenzione)
Attention mask assegna 1 ai token diversi da [PAD] e 0 a [PAD]

In [26]:
#attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
attention_mask = [1 if token != '[PAD]' else 0 for token in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 0, 0]


Ora facciamo un'altro passo che nella teoria non viene detto.
I tokens non possono rimanere parole, ma devono essere convertite in numeri perchè gli algoritmi lavorano
su questo.

In [27]:
tokens_id = tokenizer.convert_tokens_to_ids(tokens)
print(tokens_id)

[101, 1045, 2293, 3000, 102, 0, 0]


Dall'output, possiamo osservare che ogni token è mappato a un ID token univoco.
Il prossimo passo è convertite la maschera di attenzione e la lista tokens_id in tensori con pytorch
si consiglia di portarli alla gpu
Si raccomanda di fare unsqueeze(0) per aumentare le dimensioni del tensore, in quanto la semplice lista
non permette di rappresentare un vettore riga che è caratterizzato da 2 dimensioni (riga, colonna),
senza unsqueeze(0) non avremmo la rappresentazione algebrica in quanto sarebbe su un unica dimensione e rappresentebbe
solo il numero di elementi che compongono il tensore che in questo caso sarebbe solo 7
hidden_states è una lista contenete dei tensori, sono 13 tensori di dimensione (1,7,768), il
primo tensore è il embedding finale che si costruisce dalla somma di segment embedding, positional embedding e token
embedding. Gli altri 12 tensori sono le varie rappresentazioni intermedie calcolate dai vari encoder dentro il bert, poichè
questo è un bart base ci sono 12 encoder.

In [28]:
token_ids = torch.tensor(tokens_id).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

Poiché durante la definizione del modello abbiamo impostato  output_hidden_states = True
per ottenere gli incorporamenti da tutti i layer di encoder, ora il modello restituisce una tupla di output
con tre valori, come mostrato nel codice seguente. Vediamo che restituisce 3 oggetti, analizziamoli.
pooler_output è un tensore di dimensione (1, 768) che rappresenta la rappresentazione finale del token CLS, R_CLS,
viene ulteriormente elaborato da una funzione di attivazione lineare e tanh.
last_hidden_state è un tensore di dimensione (1, 7, 768) che contiene le rappresentazioni finali di tutti i token
presenti  nella lista dei tokens.
hidden_states è una lista che contiene

In [29]:
last_hidden_state, pooler_output, hidden_states = \
model(token_ids, attention_mask = attention_mask)

In [30]:
print("last_hidden_state: \n {}".format(last_hidden_state))
print("pooler_output: \n {}".format(pooler_output))
print("last_hidden_state: \n {}".format(hidden_states))

last_hidden_state: 
 tensor([[[-0.0719,  0.2163,  0.0047,  ..., -0.5865,  0.2262,  0.1981],
         [ 0.2236,  0.6536, -0.2294,  ..., -0.3547,  0.5517, -0.2367],
         [ 1.0410,  0.7755,  1.0335,  ..., -0.5621,  0.5218, -0.0852],
         ...,
         [ 0.6156,  0.1036, -0.1875,  ..., -0.3799, -0.7008, -0.3500],
         [ 0.0791,  0.4287,  0.4147,  ..., -0.2417,  0.2403,  0.0378],
         [-0.0165,  0.2459,  0.4566,  ..., -0.2179,  0.1876,  0.0228]]],
       grad_fn=<NativeLayerNormBackward>)
pooler_output: 
 tensor([[-9.0660e-01, -3.4189e-01, -3.3729e-01,  7.7140e-01,  6.0975e-02,
         -1.0525e-01,  9.0143e-01,  2.5822e-01, -2.7881e-01, -9.9997e-01,
         -1.0322e-01,  7.4772e-01,  9.8521e-01,  5.9799e-02,  9.4447e-01,
         -5.9859e-01, -2.0539e-01, -5.7386e-01,  3.7684e-01, -7.5183e-01,
          6.6604e-01,  9.9584e-01,  4.2231e-01,  2.2824e-01,  4.9139e-01,
          9.2378e-01, -6.6123e-01,  9.3111e-01,  9.6157e-01,  6.8816e-01,
         -6.7706e-01,  1.2696e-01,

In [33]:
print("last_hidden_state shape: \n {}".format(last_hidden_state.size()))
print("pooler_output shape: \n {}".format(pooler_output.size()))
print("last_hidden_state shape: \n {}".format(len(hidden_states)))

last_hidden_state shape: 
 torch.Size([1, 7, 768])
pooler_output shape: 
 torch.Size([1, 768])
last_hidden_state shape: 
 13


In [32]:
print("last_hidden_state elementi: \n {}".format(torch.numel(last_hidden_state)))
print("pooler_output elementi: \n {}".format(torch.numel(pooler_output)))
#print("last_hidden_state elementi: \n {}".format(torch.numel(hidden_states)))

last_hidden_state elementi: 
 5376
pooler_output elementi: 
 768


TypeError: numel(): argument 'input' (position 1) must be Tensor, not tuple

In [34]:
hidden_states[0].size()

torch.Size([1, 7, 768])